In [4]:
# if you're missing any of the below libraries, install here. 
import sys
!{sys.executable} -m pip install plotly


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import pgeocode
import quandl

In [6]:
# import reference df for regions
region_df = pd.read_csv(r'C:\Users\jacheung\Dropbox\Datasets\Zillow\ZILLOW-REGIONS.csv')
indicators_df = pd.read_csv(r'C:\Users\jacheung\Dropbox\Datasets\Zillow\ZILLOW_INDICATORS.csv')

la_region_id = region_df[region_df['region'].str.contains('Los Angeles', regex=False)]
la_region_id.reset_index(drop=True,inplace=True)


In [8]:
# build a look up table for los angeles
# extract out zip codes
zipcode = [k.split(';')[0] for k in la_region_id['region'] if k.split(';')[0].isdigit()]
idx = [idx for idx,k in enumerate(la_region_id['region']) if k.split(';')[0].isdigit()]
la_lookup = la_region_id.loc[idx]
la_lookup['zipcode'] = zipcode

# transform zip codes to lat long
nomi = pgeocode.Nominatim('us')
def get_lat_long(zipcode):
    vals = nomi.query_postal_code(zipcode)
    return vals['latitude'], vals['longitude']

lat_long_raw = [get_lat_long(k) for k in la_lookup['zipcode']]
lat_long_insert =np.array([np.array(xi) for xi in lat_long_raw])

la_lookup['latitude'] = lat_long_insert[:,0]
la_lookup['longitude'] = lat_long_insert[:,1]


In [9]:
# build median home values across all zip codes using quandl API
quandl.ApiConfig.api_key = 'wX4VKujcwjcKqguwo1hR'
df = pd.DataFrame()
for region_id, zipcode in zip(la_lookup['region_id'],la_lookup['zipcode']):
    data = quandl.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=str(region_id))
    data['date'] = data['date'].astype('datetime64[M]')
    data['value'] = data['value']/10**6
    group_data = data.groupby(by=['date']).aggregate('median').rename(columns = {'value':zipcode})
    df = pd.concat([df, group_data], axis=1)
    
df.reset_index(inplace=True)

scatter_df = df.set_index('date').transpose()
scatter_df = pd.concat([scatter_df, la_lookup.set_index('zipcode')], axis=1, sort=False)
scatter_df.drop(columns = ['region_id', 'region_type','region'], inplace=True)

In [14]:
import matplotlib.pyplot as plt
import plotly.express as px


# plt.scatter(scatter_df['latitude'], scatter_df['longitude'], s = scatter_df[scatter_df.columns[0]]*100)

# plt.show()


fig = px.scatter(scatter_df.reset_index(), x="latitude", y="longitude", hover_data=['zipcode'], size = )

fig.show()



In [ ]:
scatter_df[scatter_df.columns[0]]

# Plotting

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_file,show
from bokeh.models import HoverTool, CategoricalColorMapper, ColumnDataSource
from bokeh.models.widgets import Tabs, Panel

source = ColumnDataSource(df)

p = figure(title = 'SFH house price', x_axis_type = 'datetime', 
           x_axis_label = 'Date', y_axis_label = 'House price in millions (USD)')

p.circle('date', '90047', source=source, size =3, color = 'red', legend_label='Westmont')
p.circle('date', '90010', source=source, size =3,color = 'blue')
p.circle('date', '90005', source=source, size =3,color = 'blue')
p.circle('date', '90007', source=source, size =3,color = 'green', legend_label='North USC')
p.circle('date', '90006', source=source, size =3,color = 'blue', legend_label='Koreatown')
p.legend.location = "top_left"

source = ColumnDataSource(pct_df)
p1 = figure(title = 'SFH house price', x_axis_type = 'datetime', 
           x_axis_label = 'Date', y_axis_label = 'Yearly percent change in house price')
p1.circle('date', '90047', source=source, size =3, color = 'red', legend_label = 'Westmont')
p1.circle('date', '90010', source=source, size =3,color = 'blue')
p1.circle('date', '90005', source=source, size =3,color = 'blue')
p1.circle('date', '90007', source=source, size =3,color = 'green', legend_label='North USC')
p1.circle('date', '90006', source=source, size =3,color = 'blue', legend_label = 'Koreatown')
p1.legend.location = "top_left"

tab1 = Panel(child=p, title='Median home values')
tab2 = Panel(child=p1, title='Yearly percent change')

# Create a Tabs layout: layout
layout = Tabs(tabs=[tab1, tab2])
show(layout)

In [ ]:
grouped_data.set_index('date')

In [ ]:

grouped_data.plot()
plt.ylabel('Price in millions (USD)')

grouped_data.pct_change(periods=12).plot()
